In [12]:
from typing import List

class A:
    def add(self) -> List[str]:
        return [
            "a",
            "b",
        ]

class B(A):
    def add(self) -> List[str]:
        l = super().add()
        l.extend(["c", "d"])
        return l

b = B()
b.add()

<class 'super'>


['a', 'b', 'c', 'd']

In [24]:
from typing import List

class A:
    def add(self) -> List[str]:
        if hasattr(super(), "add"):
            print("has attr")
        print("no attr")

In [25]:
a = A()
a.add()

no attr


In [71]:
from functools import wraps
from types import FunctionType, ClassMethodDescriptorType

def wrapper(method, bases):
    print("wrapper!")

    @wraps(method)
    @classmethod
    def wrapped(cls, *args, **kwargs):
        print("calling wrapped!!")
        fin = []
        for base in bases:
            print("checking base: ", base)
            if hasattr(base, "add"):
                a = base.add(cls, *args, **kwargs)
                print("a add: ", a)
                fin.extend(a)
            # print("adding base!!!: ", base.add())
        r = method(*args, **kwargs)
        fin.extend(r)
        return fin
    return wrapped

class MetaClass(type):
    def __new__(meta, classname, bases, classDict):
        newClassDict = {}
        print("-----")
        print("class name: ", classname)
        print("class dict: ", classDict)
        print("bases: ", bases)
        # for base in bases:
        #     if hasattr(base, "add"):
        #         print("adding base: ", base)
        #         print("#$# base add: ", base.add())
        for attributeName, attribute in classDict.items():
            print("attributeName: ", attributeName)
            print("attribute: ", attribute)
            print("type attribute: ", type(attribute))
            if isinstance(attribute, classmethod):
                print("!!found a function!!")
                # replace it with a wrapped version
                attribute = wrapper(attribute, bases)
            newClassDict[attributeName] = attribute
        return type.__new__(meta, classname, bases, newClassDict)


class AA():
    def sub(self):
        pass

class A(AA, metaclass=MetaClass):
    @classmethod
    def add(self) -> List[str]:
        return [
            "a",
            "b",
        ]

class B(A):
    @classmethod
    def add(self) -> List[str]:
        return ["c", "d"]

class D(B):
    @classmethod
    def add(self) -> List[str]:
        return ["e", "f"]

b = D()
b.add()


-----
class name:  A
class dict:  {'__module__': '__main__', '__qualname__': 'A', 'add': <classmethod(<function A.add at 0x113b396c0>)>}
bases:  (<class '__main__.AA'>,)
attributeName:  __module__
attribute:  __main__
type attribute:  <class 'str'>
attributeName:  __qualname__
attribute:  A
type attribute:  <class 'str'>
attributeName:  add
attribute:  <classmethod(<function A.add at 0x113b396c0>)>
type attribute:  <class 'classmethod'>
!!found a function!!
wrapper!


AttributeError: readonly attribute

In [34]:
A

__main__.A

In [33]:
A()

In [47]:
class E:
    def __init__(self, l: str, a: str):
        self.a = l
    def p(self):
        print("xz")

class F(E):
    def __init__(self, l: str):
        # super().__init__(l)
        pass

f = F("x")
f.p()

AttributeError: 'F' object has no attribute 'a'

In [75]:
class X:
    def accumulate(method):
        @wraps(method)
        def wrapped(*args, **kwargs):
            r = method(*args, **kwargs)
            print("r: ", r)
            return r
        return wrapped

    def add(self) -> List[str]:
        return ["a", "b"]

In [76]:
class Y(X):
    @accumulate
    def add(self) -> List[str]:
        return ["b", "c"]
    

NameError: name 'accumulate' is not defined

In [82]:
from dataclasses import make_dataclass, is_dataclass, field
from typing import Generic, TypeVar, Type, Dict, Any
import inspect
from dataclasses_jsonschema import JsonSchemaMixin, T
from simple_parsing.helpers import Serializable

M = TypeVar("M")

class DataclassBuilder(Generic[M]):
    @classmethod
    def build(cls) -> Type[M]:
        """Generate a dataclass for a class

        Args:
            cls (Type[&quot;Server&quot;]): A class

        Returns:
            Opts: A dataclass with the init options of the class
        """
        if is_dataclass(cls):
            raise ValueError("cannot create a dataclass from a dataclass")
        sig: inspect.Signature = inspect.signature(cls.__init__)
        fin_params = []
        for param in sig.parameters:
            if param == "self":
                continue
            if sig.parameters[param].default == inspect._empty:
                fin_params.append((param, sig.parameters[param].annotation))
            else:
                fin_params.append(
                    (
                        param,
                        sig.parameters[param].annotation,
                        field(default=sig.parameters[param].default),
                    )  # type: ignore
                )

        return make_dataclass(cls.__name__ + "Opts", fin_params, bases=(Serializable, JsonSchemaMixin))

In [80]:
class Foo():

    def __init__(self, a: str):
        self.a = a

    def schema(cls) -> Dict[str, Any]


In [81]:
foo_opts = Foo.as_dataclass()
foo_opts.json_schema()

{'type': 'object',
 'properties': {},
 'description': 'FooOpts()',
 '$schema': 'http://json-schema.org/draft-06/schema#'}